# o'reillyのカサゴ深層学習の本

## ニューラルネットワークのオプション
- [勾配降下法アルゴリズムの選択](#勾配降下法アルゴリズムの選択)
- ...

## [目次](TableOfContents.ipynb)

## 参考
- https://github.com/oreilly-japan/deep-learning-from-scratch/blob/master/notebooks/

## 勾配降下法アルゴリズムの選択

### アルゴリズム
- [SGD](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92%E3%81%AE%E3%83%86%E3%82%AF%E3%83%8B%E3%83%83%E3%82%AF#o2fe1f26)
- [MomentumSGD](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92%E3%81%AE%E3%83%86%E3%82%AF%E3%83%8B%E3%83%83%E3%82%AF#afd9f568)
- [NesterovAG](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92%E3%81%AE%E3%83%86%E3%82%AF%E3%83%8B%E3%83%83%E3%82%AF#ja838935)
- [AdaGrad](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92%E3%81%AE%E3%83%86%E3%82%AF%E3%83%8B%E3%83%83%E3%82%AF#u12d8d2f)
- [RMSprop](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92%E3%81%AE%E3%83%86%E3%82%AF%E3%83%8B%E3%83%83%E3%82%AF#kdd7cbe0)
- [AdaDelta](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92%E3%81%AE%E3%83%86%E3%82%AF%E3%83%8B%E3%83%83%E3%82%AF#dc5e4977)
- [Adam](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92%E3%81%AE%E3%83%86%E3%82%AF%E3%83%8B%E3%83%83%E3%82%AF#z75dbbb3)

### 検証用の曲面の説明

#### 曲面のイメージ

In [ ]:
import numpy as np
import matplotlib.pylab as plt

# x0, x1 を -4 ～ +4 まで0.25刻み
x = np.arange(-4, 4, 0.25)
y = np.arange(-4, 4, 0.25)

# 格子点（X, Y）
X, Y = np.meshgrid(x, y)

# 格、格子点でのZ
Z = X**2 / 20.0 + Y**2

# ワイヤーフレーム図を作成
fig = plt.figure(figsize=(8, 8)) # 図の設定
ax = fig.add_subplot(projection='3d') # 3D用の設定
ax.plot_wireframe(X, Y, Z) # ワイヤーフレーム図
ax.set_xlabel('x') # x軸ラベル
ax.set_ylabel('y') # y軸ラベル
ax.set_zlabel('z') # z軸ラベル
plt.show()

#### 曲面の偏微分

##### 偏微分して勾配を計算

In [ ]:
# cf.http://d.hatena.ne.jp/white_wheels/20100327/p3
import numpy as np
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

# 偏微分（勾配を返す）
def numerical_gradient(f, X):
    h = 1e-4  # 0.0001
    
    # 要素の値が0の同じ形式のn行２列の行列
    grad1 = np.zeros_like(X)
        
    for idx1, x in enumerate(X):
        
        # 要素の値が0の同じ形式の２列のベクトル
        grad2 = np.zeros_like(x)
        
        for idx2 in range(x.size):
            tmp_val = x[idx2]
            
            x[idx2] = float(tmp_val) + h
            fxh1 = f(x)  # f(x+h)
            
            x[idx2] = tmp_val - h 
            fxh2 = f(x)  # f(x-h)
            
            grad2[idx2] = (fxh1 - fxh2) / (2*h)
            x[idx2] = tmp_val  # 値を元に戻す
        
        grad1[idx1] = grad2
        
    return grad1

# 偏微分対象の関数（曲面）
def function_3(x):
    return x[0]**2 / 20.0 + x[1]**2

# 曲面の格子点での勾配を計算
# 格子点をベクトル化して
_X = X.flatten()
_Y = Y.flatten()
# n行２列の全ペアにして（.Tは転置を意味）
koushiten = np.array([_X, _Y]).T
# 格子点での勾配を返す（.Tは転置を意味）
grad = numerical_gradient(function_3, koushiten).T

##### 結果の図示

###### 図示１

In [ ]:
# 図示 
plt.figure()
# ベクトル場を表示
plt.quiver(_X, _Y, -grad[0], -grad[1],  angles="xy",color="#666666")
# 範囲
plt.xlim([-4, 4])
plt.ylim([-4, 4])
# 凡例
plt.xlabel('x0')
plt.ylabel('x1')

plt.grid()
plt.draw()
plt.show()

###### 図示２
等高線を足してみる。

In [ ]:
# 図示 
plt.figure()

# 等高線を表示 --------------------
plt.contourf(X, Y, Z, alpha=0.5)
# ---------------------------------

# ベクトル場を表示
plt.quiver(_X, _Y, -grad[0], -grad[1],  angles="xy",color="#666666")

# 範囲
plt.xlim([-4, 4])
plt.ylim([-4, 4])
# 凡例
plt.xlabel('x0')
plt.ylabel('x1')

plt.grid()
plt.draw()
plt.show()

###### 図示３
曲面に勾配を書いてみる。

In [ ]:
# 格子点のX、Y軸方向の勾配
dX = grad[0, :].reshape(X.shape)
dY = grad[1, :].reshape(Y.shape)

# Z軸方向の勾配を計算
W = np.sqrt(dX**2 + dY**2)

fig = plt.figure(figsize=(8, 8)) # 図の設定
ax = fig.add_subplot(projection='3d') # 3D用の設定

# 曲面
ax.plot_wireframe(X, Y, Z) # ワイヤーフレーム図
# 勾配
ax.quiver(X, Y, Z, -dX/W, -dY/W, -W, 
          color='green', pivot='tail', arrow_length_ratio=0.1, length=0.5, label='grad') 

ax.set_xlabel('x') # x軸ラベル
ax.set_ylabel('y') # y軸ラベル
ax.set_zlabel('z') # z軸ラベル
plt.show()

### 実装と検証

#### 実装

In [ ]:
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
from kasago.common.optimizer import *

# 曲面
def f(x, y):
    return x**2 / 20.0 + y**2

# 勾配（微分）
def df(x, y):
    return x / 10.0, 2.0*y

# 開始視点
init_pos = (-7.0, 2.0)

# パラメタ（）
params = {}
params['x'], params['y'] = init_pos[0], init_pos[1]

# 勾配
grads = {}
grads['x'], grads['y'] = 0, 0

# 勾配降下法のアルゴリズム
optimizers = OrderedDict()
optimizers["SGD"] = SGD(lr=0.95)
optimizers["Momentum"] = Momentum(lr=0.1)
optimizers["AdaGrad"] = AdaGrad(lr=1.5)
optimizers["Adam"] = Adam(lr=0.3)

# 表のサイズなど
plt.figure(figsize=(8, 8))
plt.subplots_adjust(wspace=0.3, hspace=0.3)

# アルゴリズムを試す
idx = 1
for key in optimizers:
    optimizer = optimizers[key]
    x_history = []
    y_history = []
    params['x'], params['y'] = init_pos[0], init_pos[1]
    
    # 勾配降下
    for i in range(30):
        x_history.append(params['x'])
        y_history.append(params['y'])
        
        # 勾配
        grads['x'], grads['y'] = df(params['x'], params['y'])
        # 指定のアルゴリズムで（勾配降下
        optimizer.update(params, grads)

    # x, y を -10, -5 ～ 10, 5 まで0.01刻み
    x = np.arange(-10, 10, 0.01)
    y = np.arange(-5, 5, 0.01)
    
    # 格子点（X, Y）
    X, Y = np.meshgrid(x, y) 
    # 格、格子点でのZ
    Z = f(X, Y)
    
    # for simple contour line  
    mask = Z > 7
    Z[mask] = 0
    
    # plot 
    plt.subplot(2, 2, idx)
    idx += 1
    plt.plot(x_history, y_history, 'o-', color="red")
    plt.contour(X, Y, Z)
    plt.ylim(-6, 6)
    plt.xlim(-10, 10)
    plt.plot(0, 0, '+')
    #colorbar()
    #spring()
    plt.title(key)
    plt.xlabel("x")
    plt.ylabel("y")
    
plt.show()

#### 検証

##### SGD
勾配が最も急な向きに勾配を下る手法
- 欠点の「最短距離でない、無駄に谷間を往復する。  
学習率が大きい場合は無駄な往復が目立つ。」が顕著。
- また、このケースでは他のアルゴリズムより学習率の初期値を  
大きく設定しており、他のケースでのチューニングが難しい。

##### Momentum
[SGD](#SGD)に慣性（加速・減速）の概念を加えた最適化手法｡
- 欠点の「加速しているため、極小値付近で止まり難い。」が顕著。
- 確かに物理法則に則った様な動きをしており[SGD](#SGD)より効率的。

##### AdaGrad
- 動いた量が増えたら更新が緩やかにする。
- このブレーキ機能は学習率の減衰によって実現されている。

##### Adam
[Momentum](#Momentum)＋[AdaGrad](#AdaGrad)的な。
- 慣性（加速・減速）とブレーキ機能を持ち合わせたアルゴリズム
- AdaGradの方が効率的に見えるが、場合によっては最適解を発見できる可能性がある。